# Training Linear and Logistic Models in TensorFlow

This section ties together all the TensorFlow concepts introduced in the previous section to train linear and logistic regression models upon the toy datasets we introduced previously in the chapter.

# Linear Regression in TensorFlow

In this section, we will provide code to define a linear regression model in TensorFlow and learn its weights. This task is straightforward and you can do it without TensorFlow easily. Nevertheless, it’s a good exercise to do in TensorFlow since it will bring together the new concepts that we have introduced throughout the chapter.

# Defining and training linear regression in TensorFlow

The model for a linear regression is straightforward:

$$ y = w x + b $$

Here w and b are the weights we wish to learn. We transform these weights into tf.Variable objects. We then use tensorial operations to construct the L2 loss:

$$ \mathscr{L}(x, y) = (y - w x - b)^2 $$

The code in Example 3-11 implements these mathematical operations in TensorFlow. It also uses tf.name_scope to group various operations, and adds a tf.train.AdamOptimizer for learning and tf.summary operations for TensorBoard usage.

Example 3-11. Defining a linear regression model

In [ ]:
# Generate tensorflow graph
with tf.name_scope("placeholders"):
    x = tf.placeholder(tf.float32, (N, 1))
    y = tf.placeholder(tf.float32, (N,))
with tf.name_scope("weights"):
    # Note that x is a scalar, so W is a single learnable weight.
    W = tf.Variable(tf.random_normal((1, 1)))
    b = tf.Variable(tf.random_normal((1,)))
with tf.name_scope("prediction"):
    y_pred = tf.matmul(x, W) + b
with tf.name_scope("loss"):
    l = tf.reduce_sum((y - y_pred)**2)
# Add training op
with tf.name_scope("optim"):
    # Set learning rate to .001 as recommended above.
    train_op = tf.train.AdamOptimizer(.001).minimize(l)
with tf.name_scope("summaries"):
    tf.summary.scalar("loss", l)
    merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('/tmp/lr-train', tf.get_default_graph())

Example 3-12 then trains this model as discussed previously (without using minibatches).

Example 3-12. Training the linear regression model

In [ ]:
n_steps = 1000
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  # Train model
  for i in range(n_steps):
    feed_dict = {x: x_np, y: y_np}
    _, summary, loss = sess.run([train_op, merged, l], feed_dict=feed_dict)
    print("step %d, loss: %f" % (i, loss))
    train_writer.add_summary(summary, i)

All code for this example is provided in the GitHub repository associated with this book. We encourage all readers to run the full script for the linear regression example to gain a firsthand sense for how the learning algorithm functions. The example is small enough that readers will not need access to any special-purpose computing hardware to run.

<div style="border: 1px solid black; padding: 10px;"><b style="font-size: 2em;">Taking Gradients for Linear Regression</b><br> 
The equation for the linear system we’re modeling is y = wx + b where w, b are the learnable weights. As we mentioned previously, the loss for this system is ℒ = y − wx − b 2. Some matrix calculus can be used to compute the gradients of the learnable parameters directly for w:
$$ \nabla w = \frac{\partial \mathscr{L}}{\partial w} = -2(y - w x - b) x^T $$
and for b
$$ \nabla b = \frac{\partial \mathscr{L}}{\partial b} = -2(y - w x - b) $$
We place these equations here only for reference for curious readers. We will not attempt to systematically teach how to take the derivatives of the loss functions we encounter in this book. However, we will note that for complicated systems, taking the derivative of the loss function by hand helps build up an intuition for how the deep network learns. This intuition can serve as a powerful guide for the designer, so we encourage advanced readers to pursue this topic on their own.
        </div>

# Visualizing linear regression models with TensorBoard

The model defined in the previous section uses tf.summary.FileWriter to write logs to a logging directory /tmp/lr-train. We can invoke TensorBoard on this logging directory with the command in Example 3-13 (TensorBoard is installed by default with TensorFlow).

Example 3-13. Invoking TensorBoard

`tensorboard --logdir=/tmp/lr-train`

This command will start TensorBoard on a port attached to localhost. Use your browser to open this port. The TensorBoard screen will look something like Figure 3-8. (The precise appearance may vary depending on your version of TensorBoard.)

> **Figure 3-8: _Screenshot of TensorBoard panel._**<br><img width="300" src="data:image/png;base64, ..." />

Navigate to the Graphs tab, and you will see a visualization of the TensorFlow architecture we have defined as illustrated in Figure 3-9.

> **Figure 3-9: _Visualization of linear regression architecture in TensorBoard._**<br><img width="300" src="data:image/png;base64, ..." />

Note that this visualization has grouped all computational graph elements belonging to various tf.name_scopes. Different groups are connected according to their dependencies in the computational graph. You can expand all of the grouped elements to view their contents. Figure 3-10 illustrates the expanded architecture.

As you can see, there are many hidden nodes that suddenly become visible! TensorFlow functions like tf.train.AdamOptimizer often hide many internal variables under a tf.name_scope of their own. Expanding in TensorBoard provides an easy way to peer underneath the hood to see what the system is actually creating. Although the visualization looks quite complex, most of these details are under the hood and not anything you need to worry about just yet.

> **Figure 3-10: _Expanded visualization of architecture._**<br><img width="300" src="data:image/png;base64, ..." />

Navigate back to the Home tab and open the Summaries section. You should now see a loss curve that looks something like Figure 3-11. Note the smooth falling shape. The loss falls rapidly at the beginning as the prior is learned, then tapers off and settles.

> **Figure 3-11: _Viewing the loss curve in TensorBoard._**<br><img width="300" src="data:image/png;base64, ..." />

<div style="border: 1px solid black; padding: 10px;"><b style="font-size: 2em;">Visual and Nonvisual Debugging Styles</b><br> 
Is using a tool like TensorBoard necessary to get good use out of a system like TensorFlow? It depends. Is using a GUI or an interactive debugger necessary to be a professional programmer?
<br>
Different programmers have different styles. Some will find that the visualization capabilities of TensorBoard come to form a critical part of their tensorial programming workflows. Others will find that TensorBoard isn’t terribly useful and will make greater use of print-statement debugging. Both styles of tensorial programming and debugging are valid, just as there are great programmers who swear by debuggers and others who loathe them.
<br>
In general, TensorBoard is quite useful for debugging and for building basic intuition about the dataset at hand. We recommend that you follow the style that works best for you.
        </div>

# Metrics for evaluating regression models

So far, we haven’t discussed how to evaluate whether a trained model has actually learned anything. The first tool for evaluating whether a model has trained is by looking at the loss curve to ensure it has a reasonable shape. You learned how to do this in the previous section. What’s the next thing to try?

We now want you to look at metrics associated with the model. A metric is a tool for comparing predicted labels to true labels. For regression problems, there are two common metrics: R2 and RMSE (root-mean-squared error). The R2 is a measure of the correlation between two variables that takes values between +1 and 0. +1 indicates perfect correlation, while 0 indicates no correlation. Mathematically, the R2 for two datasets X and Y is defined as

$$ R^2 = \frac{cov(X, Y)^2}{\sigma_X^2 \sigma_Y^2} $$

Where cov(X, Y) is the covariance of X and Y, a measure of how the two datasets jointly vary, while σ X and σ Y are standard deviations, measures of how much each set individually varies. Intuitively, the R2 measures how much of the independent variation in each set can be explained by their joint variation.

<div style="border: 1px solid black; padding: 10px;"><b style="font-size: 2em;">Multiple Types of R2!</b><br> Note that there are two common definitions of R2 used in practice. A common beginner (and expert) mistake is to confuse the two definitions. In this book, we will always use the squared Pearson correlation coefficient (Figure 3-12). The other definition is called the coefficient of determination. This other R2 is often much more confusing to deal with since it doesn’t have a lower bound of 0 like the squared Pearson correlation does.</div>

In Figure 3-12, predicted and true values are highly correlated with an R2 of nearly 1. It looks like learning has done a wonderful job on this system and succeeded in learning the true rule. Not so fast. You will note that the scale on the two axes in the figure isn’t the same! It turns out that R2 doesn’t penalize for differences in scale. In order to understand what’s happened on this system, we need to consider an alternate metric in Figure 3-13.

> **Figure 3-12: _Plotting the Pearson correlation coefficient._**<br><img width="300" src="data:image/png;base64, ..." />

> **Figure 3-13: _Plotting the root-mean-squared error (RMSE)._**<br><img width="300" src="data:image/png;base64, ..." />

The RMSE is a measure of the average difference between predicted values and true values. In Figure 3-13 we plot predicted values and true labels as two separate functions using datapoints x as our x-axis. Note that the line learned isn’t the true function! The RMSE is relatively high and diagnoses the error, unlike the R2, which didn’t pick up on this error.

What happened on this system? Why didn’t TensorFlow learn the correct function despite being trained to convergence? This example provides a good illustration of one of the weaknesses of gradient descent algorithms. There is no guarantee of finding the true solution! The gradient descent algorithm can get trapped in local minima. That is, it can find solutions that look good, but are not in fact the lowest minima of the loss function ℒ .

Why use gradient descent at all then? For simple systems, it is indeed often better to avoid gradient descent and use other algorithms that have stronger performance guarantees. However, on complicated systems, such as those we will show you in later chapters, there do not yet exist alternative algorithms that perform better than gradient descent. We encourage you to remember this fact as we proceed further into deep learning.

# Logistic Regression in TensorFlow

In this section, we will define a simple classifier using TensorFlow. It’s worth first considering what the equation is for a classifier. The mathematical trick that is commonly used is exploiting the sigmoid function. The sigmoid, plotted in Figure 3-14, commonly denoted by σ , is a function from the real numbers ℝ to (0, 1). This property is convenient since we can interpret the output of a sigmoid as probability of an event happening. (The trick of converting discrete events into continuous values is a recurring theme in machine learning.)

> **Figure 3-14: _Plotting the sigmoid function._**<br><img width="300" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAzoAAAGQCAIAAADpy4JCAAAACXBIWXMAAAsSAAALEgHS3X78AAAgAElEQVR42u3dTUycZ4Lg8bc+PMxolUqNxFwGoq5YGimQlagc4sarZFzIM2ttX2xrG81tA4n6tAeD0reVDFjaWyKwtLfIAd+2RVYme+iVZzqiMvHKtPsQGCmQVa9sooa5LDMpV7TbQ7s+9vDiSqVMFcXnW8a/36EFVJnQj0vlP8/7Ps8Tq1arAcDzb3JycmpqKvw4l8stLi4aE+B0iBsCAAC5BgCAXAMAkGsAAMg1AADkGgCAXAMAQK4BAMg1AADkGgAAcg0AQK4BACDXAADkGgAAcg0AALkGACDXAACQawAAcg0AALkGAIBcAwCQawAAyDUAALkGAIBcAwDgiHNtfX19dHQ0VmdyctJ4AQCcsOSuoTY1NTU3N2d0AIDTobK1GmwX237yV+0/+WBiL/Um+oYPkmtCDQA46ZDaXGr4SnX7cXVr9dlnVr/brBZ/t+s3qX63US1uPF//x8srH5+5+otYV2p/uZbP57UaAHDg2Kps3v9BRW2tVbcff//5drGyW4e9oEO3tVq6d+PMxQ/2l2u5XK7+00wms76+bjQB4AVsr/rwqv/68ziP1bHKa/P7zrVMJpNOpwuFwsjIyMTExNzc3NTUlKEEgOdXdbtYu7BYK7D6q4rVrdXqMd+nRetEjvcM7iPXgiC4du3alStXstms4QOATk+x4kb1u40gCKrF3+188LTDzIE9L/ZstV1yzW4dANApNfZ0Yqy2UHHnoqQ7wE6LxNlL7TwtaaQAIEJhge0E2fZ3la2vAhcoXwzx7v5kGzeuyTUAOOkm27leaYasU8VSvbGXeo/3P9H1cqznx4nXhtvZxUOuAcAxZNnm/bqPlwzL/msmFevuf/br8e7Xg66Xdv0j8Z7zrb5jVyq+2zd8Xsg1ANi38B7/cLZMlv0wmwZbBVZXKt79eus/glwDgIOWWXFzp89ejIuY8e7+4Omluh9U1w+TK9bd3+YVPeQaAByByuZSuClGZXPp9JVZ/V1ZtTmt2Eu9sdQr2kuuAUDHCbfJqE2bPe+LMWv5VffBeREm1wDgeVLZXHp+4yyMsFjqldhLPU8L7OXAjV9yDQCeX9Xixk6Wba1Vtr7q/C3+fxBkT+8PMzGGXAPg9KhsrVb/z1c7cdapk2dhk4V37u/cNPac7zGBXAOAtvqso/bRCLcNi3W9HOvuq82TuWqJXAPg9KtuFyub96tbq5XNpQ6ZPwt3u4j3DO5kmaky5BoAL5rK1mpl8351a62yeT/a+8/iPYPhhFl4EdNdZcg1AF7gRNtcqmzer2wuRXWJM9yfrDZnpsyQawC86Haucm7+urJ5/+R3po1398dSr8S6+2Ld/bHUK65mItcAIMpEi6V6492vizNOSa7lcrkWnwLAAZzwhc5wkeZOn6VesTyTU5hrEg2Aw6sWNyqb9yuP/q6yef+4l3Pu9FnP4E6lpXqNP6c51wDgMMoP71Y3f11+dPdYV3TqM+QaAOxDdbtYeXj3uCfS4j2D+gy5BgD7qbTiRvnR3craJ8e0aCCW6o33nI9198V7zlscgFwDgHZVtlYrD+9WHv7tcVRavLs/3nM+1vNjU2jINQDYf6WtfXIcN6WFVznjPeftTAtyDYB9C694llc+PtpKi3f3x8/+23jPebtsgFwD4ECVtl0sfz1/tPelfX+hs+e8WTSQawAcUHltvvLo78oP7x7Jd4t1peI95+Ov/nW857x70UCuAXBwla3V8srHlYd3j2QnjnAiLd73Uys6Qa4BcCjhlmnllY+P5KJn4uwlE2kg1wA4Gkc1nRbrSsXPXoq/+teJs5eMKsg1AI5AeW2+/PUnhzxtPdzDVqWBXAPgyFS3i+WVW+WVjw8znRbrSiVeG3ZTGsg1AI401Iobpd/MlNfmD1NprniCXAPg6FU2l0oPpg9z3TNx9lL8tX+v0kCuAdBZoRbv7k8MvBs/e8l+tiDXADhi5bX50m9mDnZmVHhrWmLgXTtxgFwDoLNCLd4zmHjtp4m+YcMIcg2Ao1fZXHry2fsHCDXTaSDXADj2UDvYPWqxVG/yzTF3p4FcA+C4VIsbTz57/wChljh7KTHwbrxn0BiCXAPgeEJtu1i6d+MA+6gl+oaTb4657tlMPp+vfZzJZDKZjDFBrgGwb6UH0/s9mSDWlUoMvJsYeM91z2bm5uampqbW19drX5mYmJicnDQyyDUA9uEA6wmE2gFCDeQaAPtWLW6U7t0oP7wr1I5QPp8fHR0Vasg1AA6rtHKr/GCm/aufQq1NhUJBqyHXADiUytZq6VfvV7ZWhdpxyOVytY/DhQX1Sw0gWnFDAND5Sg+m//Bf/137rZboG/6j//A/k+fGtVqb0ul0WGmzs7OPHj26cOGCMaFzmF0D6Gj7nVSL9wyeufih7TkOYHp6OpfLpdNpQ4FcA6BdpQfTpQczbT45luo9c/FDG94e2JUrVwwCcg2Adu3rlIJYVypxbiw58J5xA7kGwEkoP7xb+uznbS7/TPQNJ9+67h41kGsAnJDSFzdKK7faeaarnyDXADhR1eLGk1/+rM1VBclzYzbpALkGwMmpbC49+eXP2rkAGu/uT/7Vh/HufoMGcg2AE1JauVX64kZb79rnxpLnxo0YyDUATs6Tz35eXpvf82nuVAO5BsBJq24Xn9z5m3ZuVrP8E+QaACetsrX65Jc/qxY3Wj8t1pVKvnU90TdsxECuAXCyrXbnb/ZcWGBVASDXACJQXpsv3buxZ6u5AArINYBoWu3JZz/f82lnLn7gAigg1wA6sdViXakzV3/hAigg1wA6sdXi3f1nfvJRLNVruAC5BtBxrZY4eyl58QM3qwFyDaAjW61v+MzFD4wVINcAOrHVHC3VaUZGRnK5XO3TTCZjTJBrAC9uq1kE2oEymYxEo3PEDQGAVgPkGsALp7K5pNUAuQbQqa22tfrklz/TaoBcA+hE1e3inueBajVArgFoNUCuAfCM0mc/r2ytajVArgF0ZKt9caP88K5WA+QaQCcqr82XVm61eEKib1irAXINIBqVrdXSvRutW80ZU4BcA4hGdbtY+tX7LZYXxLv7k29dN1CAXAOIRunejRbLC2Kp3jNXfxHrShmoekNDQzGOQj6f93KSawC0Ul6bL6/NN221rtSZn3yk1QC5BhCNPW9ZS751Pd7db6AAuQYQjda3rCUH3rMUFDi8pCEAOGCrPZhucctavGcw+bblBU1NT08XCgXjcHjZbNYgyDUAdlHZXCo9mGn2aHjLmlESGXAkXAwF2LfqdvHJZ++3eILlBYBcA4hS+cFMtbjR7NHkwHvxnkGjBMg1gGhUNpdaHDYV7+53yxog1wCiVPpiqsWjyb/60BABcg0gulZruRo0+bZd1gC5BhCdanGjvPJx0/fTnsHkwHtGCZBrAJF58lmrTXHPXHQZFJBrANEpP7xb2Vxq9mjy3Fgs1WuUALkGEJkWZ4PGu/uT58YNESDXAKJrtQfTrTZae3vCEAFyDSAy1e1iixUGib5hm+ICcg0gSqV7N5qtMIh1pZJv2RQXkGsA0akWN8pr880eTZwbczYoINcAotTiKPdYqtdGa4BcA4hSZXOpxeYdNloD5BpAxEoPppu+e/YMWmEAnIykIQDYlam150uhUJiZmfn8889rX5mens5ms0YGuQZwarW4ay3RN+wMg04LtZs3bxYKhYavd9TPmc/ngyBIp9NhRIafBkGQy+X8JSLXAPatvDbfal/cN8cMUSeHWmcaHR1dX1/PZDKPHj3K5/NDQ0NBEExMTMg19uTeNYBdlH4z07TVHA/aSQE0NTX1XLRa8HQWbX19PQiCqampIAiy2ezk5KS/R+QawL61mFqLdaUSNu/oGBcuXDiS75N/6ljL7/Lly+EHk5OT+Xw+nU7fuXPHXyJyDeAgWkytJQbetS9u56itJMjlcouLiwe+qjj01PLy8lH9bFevXq1924WFhaDuHrWbN28GQTAxMZHJZPwl0g73rgH8QGVzydTa8yKXy2UymdnZ2bCEwiuMJ6lQKMzNzX3++eeFQiGdTl++fHlkZCT8ephooYmJieDpIoPl5eVCoZDL5cbG3AGJXAM4kBZ7rZla60CPHj2K6j89OTnZsMphYWHh5s2bi4uLQRCE/1vLyiAI1tfXwxvXgrp5QZBrAPvTYq81U2vUGx0dnZube/bry8vLV69e3fXK7OjoaKFQyGQy6+vrCwsL09PThpE2uXcN4Hum1mjH+Pj4rq0Wyufzz94DNzMzEy4vuHLlSvDDmTaQawDtqmytmlpjT/l8fmbm+8UoIyMj33777aNHj9LpdP1z6v/I+vp6eF/dtWvXautDG54Dcg1gb+WVj5s9lHht2NQaofoFDblcbnZ2Np1OZzKZ+tvRGjYECS+DptPpsbGx2tM+/fRTg0mb3LsGEARBUC1ulNfmm+bawLuGiCAIlpeX62fFwiWfYZ/VX9xsSLeJiYmJiYl0Oh3OwIWrEOpn40CuAeyt1V5rTgg9LcJzn1oYHx/ftaJqp8Xfvn279sVMJlNb8hkeMFV7qH6dQTqdblh24NQp5BrAvlW3i5WHd5u+UToh9LTY83axZtvk1i5u1n+HQqEwNDT07KKBsbExM2fINYAjVl65Vd0u7vpQ4uwlU2vs2nOFQuHZ/svlcrUrpCDXAI4u177+pNlD7lo7Teq3rq1Xu0hau+jZIPxi6yOq0un0tWvXHNmOXAM4hlZrfqB7vGcw3jNoiE6NPW8ay2azLZ7TsN4zm82Ga0J/9KMfZbPZcEM1kGsAx5Brrfbv+KnxoZnp6WmLBjgZ9l0DXmiVzaXK1uquD8VSvYm+YUNETSaTqf+09bVRkGsAR8Nda+wr1+qXfE5NTTWsPGhxMhXINYCDaLE1bqwrlXjN1BqN6m9QKxQKb7zxxtBTf/qnfzo6OmqIkGsAR6n8dfNjDJw6xW6e3aQj/1TtU6OEXAM4ulxrscjAlVB2k8lkZmdnjQMnzMpQ4EVttbV5W+OeMg1bprV/tEBtgWc7f2RkZCSXy01NTeXz+dp5BrlcLpvNXrhwwVpR5BrA0eWaqbVTZ3p6+mB/sNn2uc2YY+OEuRgKvIha799ha1xArgFErMX+HQ50B+QaQMRa798RP3vJEAFyDSBK9u8A5BpAZ+eaRQaAXAPo3Fazfwcg1wA6OtccEgrINYCOVdlarWwu7fqQ/TsAuQYQvRZ3rdm/A5BrABGrbhft3wHINYDOVV651ewh+3cAcg2gA3LNIgNArgF0bqutzVeLG7u3mv07ALkGEH2u2RoXkGsAHauyuVTZWt31Ift3AHINIHot7lqzfwcg1wAiVi1utNi/I9E3bIgAuQYQJXetAXINoHNVt4vlr+eb5tprptYAuQYQqfLKrep2cfdW6xu2fwcg1wCizjVb4wJyDaBzW6351rjxnsF4d78hAuQaQJRKv5lp9lDy3LjxAeQaQJTKD+82m1qzNS4g1wA6INea799ha1zg+ZI0BMDpk/q/a5XNh7s+FEv12hoXeL6YXQNOof94vulDFoQCcg0gYn/5F3/8l3/RtetDsa6UrXEBuQYQsf/0715u9lBi4N1YV8oQAXINIDJ7TK0NvGeIALkGECVTa4BcA+hcptYAuQbQ0UytAaeSfdeAU+Iv/+KP/823ptaAU8jsGnBKvJX4vNlDptYAuQYQsfLafGVrddeHTK0Bcg0geqXfzDR7yNQaINcAom61lVvV4sauD5laA+QaQMSq28XyA1NrgFwD6FTllVvV7eKuD5laA+QaQMSq28XyysfNHk2+dd3UGiDXAKLUamot1ZvoGzZEgFwDiEy1uFFqftda8s0xQwTINYAotdi8o/iv+kytAXINIErV4kZ5bb7Zo//lvhEC5BpApFpMrf39b7e/+O22IQLkGkBkWk+t/ef/8dgQAXINIEqtp9b+/rf/YogAuQYQGVNrgFwD6Gim1gC5BtC5qtvFysO7zR41tQbINYCItTjGwNQaINcAItb6hNB7v88aIkCuAUSpxdRavGcw/ueDhgiQawCRaT21ljw3bogAuQYQpcrDu62m1npMrQFyDSBSLfbvSAy8a3wAuQYQpfLafLW4setDsVRv4uwlQwTINYAotZhaS745ZnwAuQYQpcrmUquptb5hQwTINYAolR5MN3vI1Bog1wAiVtlarWwu7fpQrCsVd9caINcAotVir7XEwLuxrpQhAuQaQGSqxY3y2nzzXHvPEAFyDSBKrabW+oZNrQFyDSBK1e1i+eumU2sWGQByDSBiLQ50T/QNx1K9hgiQawCR5trXnzR7KPHaT40PINcAIm215qdOOdAdkGsA0XOgO4BcAzrXHqdO2RoXkGsA0XLqFIBcAzqXU6cA5BrQ0Zw6BSDXgM7l1CkAuQZ0NKdOAcg1oHM5dQpArgEdrdWpU2cvOXUKkGsAkedaq0UGxgeQawCRttrafLOpNadOAXINIHpOnQKQa0DnanGgu1OnAOQaEL0WU2sWhALINSBi5Yd3W02t9Q0bIkCuAUSaaxaEAsg1oGNVNpdaHOieeM3UGoBcAyJVejDd7CEHugPINSBie0ytOdAdQK4B0TK1BiDXgM5lag1ArgEdzdQagFwDOld5bd7UGoBcAzpX6xNCTa0ByDUgSq1OCDW1BiDXgGhVt4um1gDkGtC5yiu3TK0ByDWgQ1W3i61PCDW1BiDXgCiV7t2obhd3fSiW6jW1BiDXgChVtlbLa/PNHk2+OWZqDUCuAVEqfTHV7KFYqjfRN2yIAOQaEJkW++IGQXDm4oeGCECuAZGpbhdL9240fRvqGYz3DBolALkGRKb8YKbZCoPA1BqAXAOiVdlaLa3cavZocuC9WKrXKAHINSAypV+93+yhWFcqcW7MEAHINSC6VnswXdlabfZo8q3rNu8AkGtAZCpbq6UHTY8HjfcM2rwDQK4BUWpxGTQIguTbE4YIQK4B0bVa68ug58bi3f1GCUCuAdFofRnU8aAAcg2IUnW7+OSXP2vxhDMXP7TCAECuAZEp3btRLW40ezQ58J4zDADkGhCZ8tp8eW2+2aOxVK+N1gDkGhCZytZqi7NBgyA485OPXAYFkGtANMJb1lqcDWo1KIBcA6L05Jc/a3HLWrxnMHlu3CgByDUgGqUvblQ2l5o9GutKnbn4oVECkGtANMpr86WVWy2ekLz4QSzVa6AA5BoQgcrm0pPPft6q1QbeS5y9ZKAA5BoQRattrbbeETfeM5h8+7qBApBrQASq28Und/6mxVLQWKr3zE8+MlAAcg3oyFbrStllDUCuAVG2WmVrtcVzkm9dt8sagFwDOrXV3r6e6Bs2VgByDejEVkv0DScH3jNWAHIN6MhWO3vpzMUPjBWAXAM6sdXi3f1JrQYg14CObbUzV39hKSjAkUsaAqC1ytZq6z07tBrAsTK7Bmg1gI5mdg1oqrw23/o8UK0GINeAyJS+uFFauaXVAOQa0HGq28XSZz8vP7yr1QDkGtBxKlurpV+933oRqFYDkGtANMpr86V7N1ovLAiCIHH2UvLiB1oNQK4BJ6e6XSzdu1Fem9/zmYm+YecWAMg14ES1eQE0CILk29edBwog14ATVVq5Vfrixp5Pi3Wlkm9dT/QNGzEAuQackGpx48ln71c2l/ZutVTvmZ98FO/uN2gAcg04IaWVW+UHM3uuKgiCIN4zeOYnH1lYACDXgBPS/qRaEATJgfeSb183aAByDTghpQfTpQcz7Twz1pVKXvwgcfaSQQOQa8BJqGwuPfns/Wpxo50nx7v7z/zko1iq17gByDXg2FWLG6V7N/Y8VOr794VzY8lz48YNQK4Bxx9q28Xyyq02r34G4QrQix/GewYNHYBcA45d+2s/d94OBt5LnBuzAhRArgHHrrw2X/rNTJu3qQUm1QDkGtCxoRaYVAOQa0DHhlq8uz/5Vx86qwBArgEdF2qxrlTi3JjD2gHkGnCMqtvFysO7+w21IAgSfcPJt667+gkg14BjDLXyyq3yysftr/oMxXsGk+fGLSkAkGvAcalsrZZXPi6vze/3D8ZSvcm3rjtRCkCuAcelvDZf/vqTNs9l/0GodaWSb11P9A0bQwC5Bhy9anGjvPJx+ev5/V73DEMtMfBuYuA9t6kByDXgqCttu1h5ePdg02lCDUCuAceosrlU/vqTysO7B5hOE2oAcg04LjsXPR/d3e+uHN+HWqo3MfBu4rVhoQYg14AjrbRHdytrn1S2Vg/8TWKp3uSbYxYTAMg1oLMqLbCPGoBcA45WZWu1snn/8JUW60rFz15KvjkWS/UaVQC5Bhy60jaXKg//9jD3pdXEu/sTA+/Gz15ygxqAXAMOJdyJo/KPvz7wGs964XRa4rWfuu4JINeAQ6lsrVYe3q08/NtDXu6sMZ0GgFyDwwrXDVQ3f13ZvH/4ibRQLNWbePVSYuBdd6cBINfggIlW2bxf+cdfVzbvH/6OtO8rzUVPAOQadFqi1Sot/upfJ85eMs4AyDXYh8rmUmXrq6O90KnSAJBrcHDV4sZOn219dbDj1feutFRvvOe8SgNArkF7fbZdrG6tVjbvV7fWKltfHe1Vznrx7v54z/l430/j3f2GHQC5BtH3WShx9lKs58eJVy9Z4wmAXIMmfRZe39xaPZk+C55OpMV6fuxyJwByDZ6Js3DybOuroLh5fPefPWtn3cCf/zjec95EGgByDXaPs+rW6pGv39wj0XrOx3p+HO857440AOQaBJWt1WC7WNm8X/1us1r83QnHmUQDQK7BLmUWbH9X2fqq+t3GCdxz1ky8uz/2Z6+70AmAXONFFF7NrG4/rm6tdkKZhWJdqVh3f7xnMBYuGnC8OgByjRcwy4LtYmVrtXN+wnjPYLz79Vh3X+zPXneVEwC5xmlusiAIKpv3gyCobq1Vtx93wmxZsz6LpV6JdffFu193pDoAco3TFmTV4u+q320EQRDulBHJXf/7snN90/wZAHKNU1Jj4VXLIAjXYD4XQdYg3t2/M3nWcz72Uq8lAgCcklybm5u7efPm8vJyEASZTCaXy01MTGQyGSN1CtQ2jK1NjNVSrNNuIztMnMW6+2OpV0yeAXAKc61QKFy9ejWfz9e+sr6+Pjc3t7CwsLi4mM1mDVYnF1jw9HaxxjLr1PvGDiO8rBlLvRJ7qSfecz7oSokzAF6IXGtotfqMGxoaevToUTqdNl7HpHYJ8umnj3/waW0O7JlEexHEewZjXS/HuvtiL/XGUq/EuvttqwHAi5hrc3Nzu7ZardjGx8dnZ2eNV4vGqsup+43P/GFvBc/hfWAnIJbq3Qmyl3pi3f2xrpeVGQB8n2tTU1P1X52dnU2n06Ojo4VCodZz09PTUU2wNQujdoT7e+3xpHAbsGb/9dN4PbFDsizoSsW7X3c1EwD2yLXl5eX19fXal8bGxkZGRsKv12fcwsJC+PX2lR/eLd27oXVexCbrSsW6+4MgCPcz25ktszwTAA6WawsLC/Vfunz5cvjBlStX6nPt888/31eulVZulb64YYhPsZ0UC+fJgiDecz4IAk0GAEcrVq1Wh4aG6m9cq1ar3z8ci9U+zmazX3755S7fou45NS//Sfx/Tf35y38SN8TPr3/Y+EPh99UgCL743/8SBEHh/1X+YfNJEATf/FPpm38uGR8AOJjFxcVcLtf+83cuhtY+b7g7LZPJ1K6T1j9tTwO9f6TVOtnf/3Z7p8k2//D495UgCFY2/vD499WnH1QMEQB0iGQQBLX1BEEQNOyvVp9r4TNt59Gxvvnn0jf/VG6IsPoyMysGAM9lru1rzmx5ebnNubtv/kkWHEqtsYIgePz7yj9s/qH2aW0aTIEBwAuRa/VTawezuLi469e/++6/v/TNf3sBxzS8Ab9e7Wb877/yUm8s9coP++xf6j/9o3/9/cd/FgQX6x666GULu7l9+/bc3Fz4cTabnZ6eNiZAZ9rvYVFHcMR78/m2XOmLdGnlVqfF07PCTfObPvpMWu040t3Ccj1evXAo9Uum0un0vm7jBehkyeP97m9fT5wbqx767HC72wMAcq0tB1hnEOtKxdqY3wIAYFfxfV0v2O+lVgAADptrDZ83rDyo38UDAIBocq1+gq1hX4/6XHPfLgBANLnWcImzVmwN6eZKKABANLl24cKF+i/VTnyvXxX/7NMAADgBySAIrly5Un/Y1M2bNzOZTDqdnpqaqj0vk8lcuXLFeAEAnLCdpQbvvPNO7UuFQmF0dPTq1av1yw4mJiYMFgBAZLk2OTnZ4ta0XC43MjJisAAAIsu1IAju3Lmza7Fls9k7d+4YKQCAiHMtk8ksLi5OTExkMplaqE1MTHz55ZcHOMwAAIAj8YNDqNLp9OTk5OTkpHEBAOjEXAMA4OQVCoVwv9tMJlO7znmoXMvn8zdv3qytG81ms9PT082ePDc3d/PmzdpPkMvl6q+3QsOr5fbt280ebf1KAzi85eXl8fHxFk9YXFw0SrTTXjMzM/WxlMvlmr14CoXC+Pj43Nxc7SuZTGZ6erp+A7X95Vo+n5+ammrYPrfFz3r16tX6J6+vr8/NzS0sLCwuLjojgWetr6+3+eoCOKZ/Zb0LcbShtudvCENDQ8+e2H716tXZ2dnavhzx9kNtaGhoaGio/ddxQ6vV/z959ieDIAi++eYbgwBE+0ujQeDAoTY5Ofnqq69OTU21GTnhxFazJ4+OjtaOA20r18L029cvHHNzcy2eH877+avFGyXgXYhTk2vth1poZmam9UuuFkttXQzNZrPpdHpfP0H9AVZBEMzOzqbT6dHR0do3mZubm56etkUIDa/12scjIyM/+tGP6h91yyNw3B4/flz7OJfLOSyb9oVLBGr5Vf9xMzdv3qx9nE6nw63T3njjjdofzOfz6+vrmUym3XvXstlsPp9Pp9PXrl3LZrNXr15t8eTl5eX6H3FsbCy8+Lq8vFyfcQsLCw5LoOGVU/vYkhQg2nehd955xz9S7Es2m11fXw9XVQZBMDQ01OLJCwsLDU/SAkYAAAQ/SURBVKd9hv/qTUxMjI6O1j9tbGys3XvXLly4MDY29ujRo8nJyT2nxBYWFuo/vXz5cvhBwyHxn3/+ub9aWvyaYhAA70I8Ry5fvnznzp3FxcVcLrfnkxsqqPZHGv5s+LR2Z9f2tXdus5+gYTVo/S8xUH+zo3dJwBsRz519Tcc2VFCtkRpeeOHT4sfx49b/BA1TcfU/hFzDL7WANyL8btDwYquf3grvLjuWXKu/Ftswo9bwA9nOg13zPZ/Px2KxN954Y3x83EIt4OTfhYIgiMVir7766ujoqJ3YOOHfDRqmutbX1+PH/XI/widzuj3b7svLyzMzM6+++qptX4BI3oXC3d3DbUfNL3CE9vU7wLHkmhc0B1O/fr7BzMxM/TIZgJPJtfp/XBUbEdpZarC8vPzsqzCdTjsqiqOy52tsYGAgXPn8zTffhDvN1D9zbm7unXfeaWetDcDBpNPp8F3o8ePH+Xy+4fpPON+/r4V3cMS5Nj4+/uy8XIvjSGG/9nyNNSyoWVhYqN9XOQiCqakpuQYcn1wuV/8ms7y8XH8KUPguJNeIRDza/7xTDWjmypUrs7Oz9V9xty9wkrLZ7J07dxr+nfJGxMn3TyaTOfpc29f8h4uttC62XbefATgZmUymYYN370Ic1S8D+3od7lwMnZ6e3vW+osP/QA3f1qYML6yDvcYazlxzny9wwhoOL/YuxFGpP429oY6e3b82eYDK21Mul6tNFzf8IlL/A7kPyW8SB3hxG0kg2n9iDQJH9c9iLZYacu3Z/Wvjx/QT7BqJzc5bgF0VCoWG3zC8ZoAT1uxYRTikCxcu1H9aS7eG+yPDp8VP4Ceonfi+608AzczMzNT/huFdEjhh+Xy+/l8uvzRyhBpui7x9+3b4waeffvrs05LH9BPU33J08+bNTCaTTqenpqZqz3n2/k1eZMvLy7dv3758+XLYZOFO4vUvmCAIrl27ZqBoYWRkpNb0rlhxAKOjo+G7UHhT0cLCQsORKt6FOELZbDabzdYuIs3NzYXvXTMzM7Xn5HK5nd8QqvvXsBlbLpd79jnhToMtzM7OVuGp6enpPX8HMErA8fnyyy9bvwtlMplvv/3WQHGEsXTnzp3Wr7rFxcXwmce179rk5GSLGeNcLtewJyovuJWVlda/gjTswQZw5HP8LR5Np9PP7sEGh3TlypUWOTQ2Nla7YnCM2+TeuXNn12ILNx70l0Sbb5QjIyOLi4veJYGofmkMz19x1xrHYXp6etdiGxsbq7/uFKtWqwf4l7X+cn42m212JatQKMzMzNy+fTu8jy2bzV6+fNkJHuz6orp9+/by8nLtaNHwgv21a9cadsoFOA7r6+sLCwuffvrp+vp67d+sbDbrtGIO/IoKb0cLZTKZFhNp+Xy+IZYaXnUHyTUAAE5M3BAAAMg1AADkGgCAXAMAQK4BACDXAADkGgAAcg0A4JT7/yqRLURjmDheAAAAAElFTkSuQmCC" />

The equations for predicting the probabilities of a discrete 0/1 variable follow. These equations define a simple logistic regression model:

$$ y_0 = \sigma (w x + b) $$

$$ y_1 = 1 − \sigma (w x + b) $$

TensorFlow provides utility functions to compute the cross-entropy loss for sigmoidal values. The simplest of these functions is tf.nn.sigmoid_cross_entropy_with_logits. (A logit is the inverse of the sigmoid. In practice, this simply means passing the argument to the sigmoid, wx + b, directly to TensorFlow instead of the sigmoidal value σ wx + b itself). We recommend using TensorFlow’s implementation instead of manually defining the cross-entropy, since there are tricky numerical issues that arise when computing the cross-entropy loss.

Example 3-14 defines a simple logistic regression model in TensorFlow.

Example 3-14. Defining a simple logistic regression model

In [ ]:
# Generate tensorflow graph
with tf.name_scope("placeholders"):
  # Note that our datapoints x are 2-dimensional.
  x = tf.placeholder(tf.float32, (N, 2))
  y = tf.placeholder(tf.float32, (N,))
with tf.name_scope("weights"):
  W = tf.Variable(tf.random_normal((2, 1)))
  b = tf.Variable(tf.random_normal((1,)))
with tf.name_scope("prediction"):
  y_logit = tf.squeeze(tf.matmul(x, W) + b)
  # the sigmoid gives the class probability of 1
  y_one_prob = tf.sigmoid(y_logit)
  # Rounding P(y=1) will give the correct prediction.
  y_pred = tf.round(y_one_prob)
with tf.name_scope("loss"):
  # Compute the cross-entropy term for each datapoint
  entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_logit, labels=y)
  # Sum all contributions
  l = tf.reduce_sum(entropy)
with tf.name_scope("optim"):
  train_op = tf.train.AdamOptimizer(.01).minimize(l)
     train_writer = tf.summary.FileWriter('/tmp/logistic-train', tf.get_default_graph())

The training code for this model in Example 3-15 is identical to that for the linear regression model.

Example 3-15. Training a logistic regression model

In [ ]:
n_steps = 1000
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  # Train model
  for i in range(n_steps):
    feed_dict = {x: x_np, y: y_np}
    _, summary, loss = sess.run([train_op, merged, l], feed_dict=feed_dict)
    print("loss: %f" % loss)
    train_writer.add_summary(summary, i)

# Visualizing logistic regression models with TensorBoard

As before, you can use TensorBoard to visualize the model. Start by visualizing the loss function as shown in Figure 3-15. Note that as before, the loss function follows a neat pattern. There is a steep drop in the loss followed by a gradual smoothening.

> **Figure 3-15: _Visualizing the logistic regression loss function._**<br><img width="300" src="data:image/png;base64, ..." />

You can also view the TensorFlow graph in TensorBoard. Since the scoping structure was similar to that used for linear regression, the simplified graph doesn’t display much differently, as shown in Figure 3-16.

> **Figure 3-16: _Visualizing the computation graph for logistic regression._**<br><img width="300" src="data:image/png;base64, ..." />

However, if you expand the nodes in this grouped graph, as in Figure 3-17, you will find that the underlying computational graph is different. In particular, the loss function is quite different from that used for linear regression (as it should be).

> **Figure 3-17: _The expanded computation graph for logistic regression._**<br><img width="300" src="data:image/png;base64, ..." />

# Metrics for evaluating classification models

Now that you have trained a classification model for logistic regression, you need to learn about metrics suitable for evaluating classification models. Although the equations for logistic regression are more complicated than they are for linear regression, the basic evaluation metrics are simpler. The classification accuracy simply checks for the fraction of datapoints that are classified correctly by the learned model. In fact, with a little more effort, it is possible to back out the separating line learned by the logistic regression model. This line displays the cutoff boundary the model has learned to separate positive and negative examples. (We leave the derivation of this line from the logistic regression equations as an exercise for the interested reader. The solution is in the code for this section.)

We display the learned classes and the separating line in Figure 3-18. Note that the line neatly separates the positive and negative examples and has perfect accuracy (1.0). This result raises an interesting point. Regression is often a harder problem to solve than classification. There are many possible lines that would neatly separate the datapoints in Figure 3-18, but only one that would have perfectly matched the data for the linear regression.

> **Figure 3-18: _Viewing the learned classes and separating line for logistic regression._**<br><img width="300" src="data:image/png;base64, ..." />